In [1]:
KotlinVersion.CURRENT             // 코틀린 현재 버전 

1.8.0

In [2]:
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-core:1.6.4")

In [3]:
:classpath

Current classpath (16 paths):
/Users/a06411/opt/anaconda3/envs/kotlin/lib/python3.10/site-packages/run_kotlin_kernel/jars/lib-0.11.0-170.jar
/Users/a06411/opt/anaconda3/envs/kotlin/lib/python3.10/site-packages/run_kotlin_kernel/jars/api-0.11.0-170.jar
/Users/a06411/opt/anaconda3/envs/kotlin/lib/python3.10/site-packages/run_kotlin_kernel/jars/kotlin-script-runtime-1.8.0-dev-3517.jar
/Users/a06411/opt/anaconda3/envs/kotlin/lib/python3.10/site-packages/run_kotlin_kernel/jars/kotlin-reflect-1.7.10.jar
/Users/a06411/opt/anaconda3/envs/kotlin/lib/python3.10/site-packages/run_kotlin_kernel/jars/kotlin-stdlib-1.7.10.jar
/Users/a06411/opt/anaconda3/envs/kotlin/lib/python3.10/site-packages/run_kotlin_kernel/jars/kotlin-stdlib-common-1.7.10.jar
/Users/a06411/opt/anaconda3/envs/kotlin/lib/python3.10/site-packages/run_kotlin_kernel/jars/annotations-13.0.jar
/Users/a06411/opt/anaconda3/envs/kotlin/lib/python3.10/site-packages/run_kotlin_kernel/jars/kotlinx-serialization-json-jvm-1.3.3.jar
/Users/a06

In [4]:
import kotlinx.coroutines.*

# 1. 코루틴 알아보기

## 1-1 코루틴 처리 

In [5]:
import kotlinx.coroutines.*                               // 코루틴 패키지

suspend fun createCoroutines(amount:Int)= runBlocking { 
    var jobs = ArrayList<Job>()                           // 배열의 작업 결과를 저장
    for (i in 1..amount) {
        jobs += launch {                                  // 코루틴 처리 후 결과 저장 
            print("...")
            delay(1000)
        }
    }
    
    jobs.forEach {
        it.join()                                         // 코루틴 실행할 때까지 기다린다. 
    }
    println()
    println(jobs)                                         // 코루틴 처리 결과 
}

fun main() = runBlocking {
    println("코루틴 처리 : " + coroutineContext)             // 코루틴 환경구성 
    print("start")
    createCoroutines(10)                                  // 코루틴 일시중단함수 실행 
    println("end")
    println("코루틴 처리 : " + coroutineContext)             // 코루틴 환경구성 
}

In [6]:
main()

코루틴 처리 : [BlockingCoroutine{Active}@3da29192, BlockingEventLoop@195b04f0]
start..............................
[StandaloneCoroutine{Completed}@319636b5, StandaloneCoroutine{Completed}@6fd43414, StandaloneCoroutine{Completed}@3728b62c, StandaloneCoroutine{Completed}@57efcc7d, StandaloneCoroutine{Completed}@574c6bab, StandaloneCoroutine{Completed}@197681c2, StandaloneCoroutine{Completed}@6b9dcd16, StandaloneCoroutine{Completed}@42b8683c, StandaloneCoroutine{Completed}@769d52f3, StandaloneCoroutine{Completed}@5b48a573]
end
코루틴 처리 : [BlockingCoroutine{Active}@3da29192, BlockingEventLoop@195b04f0]


### 일반함수와 일시중단함수를 코루틴에서 처리

In [7]:
fun task1() = println("일반함수 task 1 : " 
                     + Thread.currentThread().name)       // 첫번째 함수 

fun task2() = println("일반함수 task 2 : "                  // 두번째 함수 
                     + Thread.currentThread().name)

In [8]:
suspend fun task3() =  println("일수중단함수 task 3 : "      // 일시중단함수 정의
                     + Thread.currentThread().name)

val task4 = suspend {println("일시중단함수 task 4 : "        // 람다표현식도 일시중단함수 정의 가능 
                     + Thread.currentThread().name)}

In [9]:
val rb = runBlocking { 
    println("코루틴 처리 : " + coroutineContext)             // 코루틴 환경구성 
    launch {                                              // 코루틴 빌더로 함수처리
        delay(300)
        task1()                                           // 일반함수 호출 
    }
    launch {                                              // 코루틴 빌더로 함수처리
        task2()                                           // 일반함수 호출 
        delay(300)
    }
    launch {                                              // 코루틴 빌더로 함수처리
        delay(300)
        task3()                                           // 일시중단함수 
    }
    launch {                                              // 코루틴 빌더로 함수처리
        task4()                                           // 일시중단함수
        delay(300)
    }
}                                                         // 메인 스레드 사용

코루틴 처리 : [BlockingCoroutine{Active}@326ec8e7, BlockingEventLoop@75d78b75]
일반함수 task 2 : Thread-33
일시중단함수 task 4 : Thread-33
일반함수 task 1 : Thread-33
일수중단함수 task 3 : Thread-33


## 1-2 디스패처 알아보기¶

- Dispatcher는 CoroutineContext를 상속받아 어떤 쓰레드를 이용해서 동작할 것인지 정의해둔 것 입니다.

- None : 실행된 코루틴 스코프에게 상속받는다.
- Unconfined : 호출 쓰레드에서 실행하지만 멈췄다가 재개할 경우 재개한 쓰레드에서 실행한다.
             -> 특정 쓰레드에 국한되지 않는 경우 적합

- Main : 메인 쓰레드에서 실행한다.
         -> 안드로이드에서 UI를 바꿀 때 적합, Job이 무거우면 부적합

- IO : 네트워크, 디스크 사용 할때 사용합니다. 파일 읽고, 쓰고, 소켓을 읽고, 쓰고 작업을 멈추는것에 최적화되어 있습니다.
- Default : Thread Pool에 할당한다. (GlobalScope.launch { }도 마찬가지)

In [10]:
fun main() { 
    runBlocking {
        launch {
            println(" 미지정 일때  : " + coroutineContext)             // 코루틴 환경구성 
            println(" launch(미지정) : ${Thread.currentThread().name}")
        }

        launch(Dispatchers.Unconfined) {
            println("Unconfined : " + coroutineContext)             // 코루틴 환경구성 
            println("launch(Unconfined) : ${Thread.currentThread().name}")
        }

       //launch(Dispatchers.Main) {   //안드로이드 용
       //    println("launch(Dispatchers.Main) : ${Thread.currentThread().name}")
       //}

        launch(Dispatchers.IO) {
            println(" IO : " + coroutineContext)             // 코루틴 환경구성 
            println("launch(IO) : ${Thread.currentThread().name}")
        }

        launch(Dispatchers.Default) {
            println("Default : " + coroutineContext)             // 코루틴 환경구성 
            println("launch(Default) : ${Thread.currentThread().name}")
        }
    }
}

main()

Unconfined : [StandaloneCoroutine{Active}@558f38f9, Dispatchers.Unconfined]
launch(Unconfined) : Thread-37
 IO : [StandaloneCoroutine{Active}@423608be, Dispatchers.IO]
 미지정 일때  : [StandaloneCoroutine{Active}@18204ca8, BlockingEventLoop@169e61c8]
Default : [StandaloneCoroutine{Active}@7c6c3b2d, Dispatchers.Default]
launch(IO) : DefaultDispatcher-worker-1
launch(Default) : DefaultDispatcher-worker-3
 launch(미지정) : Thread-37


In [11]:
fun main() = runBlocking {                                     // 런블러킹 컨텍스트 확인 
    println("runBlocking : ${Thread.currentThread().name}")    
    println("컨텍스트 : " + coroutineContext)                                      
    println("상태 : " +isActive)                                // 잡상태 확인
    
    val g = GlobalScope.launch {                               // 전역스코프 컨텍스트 확인 
        delay(300)
        println("전역스코프 : ${Thread.currentThread().name}")
        println("컨텍스트 : " +coroutineContext)
        println("상태 : " +isActive)                            // 잡상태 확인 
    }
    println("전역스코프 반환 : " + g)
    val c =CoroutineScope(Dispatchers.Default).launch {        // 코루틴스코프 컨텍스트 확인
        delay(500)
        println("코루틴 스코프 : ${Thread.currentThread().name}")
        println("컨텍스트 : " +coroutineContext)
        println("상태 : " +isActive)                             // 잡상태 확인    
    }
    println("코르틴 스코프 반환 : " + c)
    delay(1000)
}

main()

runBlocking : Thread-44
컨텍스트 : [BlockingCoroutine{Active}@7810f788, BlockingEventLoop@4a7562ce]
상태 : true
전역스코프 반환 : StandaloneCoroutine{Active}@7a40ce5d
코르틴 스코프 반환 : StandaloneCoroutine{Active}@538880b8
전역스코프 : DefaultDispatcher-worker-3
컨텍스트 : [StandaloneCoroutine{Active}@7a40ce5d, Dispatchers.Default]
상태 : true
코루틴 스코프 : DefaultDispatcher-worker-3
컨텍스트 : [StandaloneCoroutine{Active}@538880b8, Dispatchers.Default]
상태 : true


In [12]:
fun main() {
    Thread(Runnable {                                   // 3개의 러너블 객체를 만들어서 스레드 처리               
        for (i in 0..2) {
            println("쓰레드 처리  : $i "                   // 3개의 쓰레드를 처리 
                    + Thread.currentThread().name)
            Thread.sleep(500L)                          // 스레드 중단 : 현재 스레드 일시중단 
        }
    }).start()                                          // start 메소드로 스레드 시작

    GlobalScope.launch {                                // 코루틴 전역스코프로 처리 
        repeat(3) {                                     // 3개의 일지중단함수 만들고 코루틴 생성 
            println("코루틴처리   : $it " 
                    + Thread.currentThread().name)      // 코루틴 내의 dispatchre로 처리 
            delay(500L)                                 // 일시 중단 다른 작업 수행 
        }
    }
    
    Thread.sleep(3000L)                                 // 어플리케이션 중단 
}

main()

쓰레드 처리  : 0 Thread-51
코루틴처리   : 0 DefaultDispatcher-worker-3
쓰레드 처리  : 1 Thread-51
코루틴처리   : 1 DefaultDispatcher-worker-3
쓰레드 처리  : 2 Thread-51
코루틴처리   : 2 DefaultDispatcher-worker-3
